In [10]:
#-*- coding: utf-8 -*-

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from pandas import DataFrame, Series
from sklearn.preprocessing import LabelEncoder

%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [11]:
phone_brand_device_model = pd.read_csv('data/phone_brand_device_model.csv')
gender_age_train = pd.read_csv('data/gender_age_train.csv')
label_categories = pd.read_csv('data/label_categories.csv')
events = pd.read_csv('data/events.csv')
app_labels = pd.read_csv('data/app_labels.csv')
app_events = pd.read_csv('data/app_events.csv', dtype = {'event_id' : np.int8, 'app_id' : np.int64, 
                                                   'is_installed' : np.int8, 'is_active' : np.int8})
gender_age_test = pd.read_csv('data/gender_age_test.csv')

### 명목형 변수 치환

In [12]:
le_br = LabelEncoder()
le_mo = LabelEncoder()


phone_brand_device_model['brand'] = le_br.fit_transform(phone_brand_device_model['phone_brand'])
phone_brand_device_model['device'] = le_mo.fit_transform(phone_brand_device_model['device_model'])



phone_brand_label = phone_brand_device_model.copy()

phone_brand_label = phone_brand_label.drop('phone_brand', 1)
phone_brand_label = phone_brand_label.drop('device_model', 1)

#M = 1, F = 0
le_gen = LabelEncoder()
gender_age_train['M_F'] = le_gen.fit_transform(gender_age_train['gender'])

#group 치환
le_g = LabelEncoder()
gender_age_train['group_e'] = le_g.fit_transform(gender_age_train['group'])

# evnet_id 에서 minus값 제거, memory 사용량 줄이기

app_events = app_events[(app_events.event_id > 0)]


### Data set 기초자료 (train + test)

In [16]:
tmp_train = pd.merge(gender_age_train, phone_brand_label, how='inner', on = 'device_id')
tmp_test = pd.merge(gender_age_test, phone_brand_label,  how='inner', on = 'device_id')

tmp_train = tmp_train.drop_duplicates(subset=['device_id'])
tmp_test = tmp_test.drop_duplicates(subset=['device_id'])

con = pd.concat([tmp_train, tmp_test])

tmp = pd.merge(con, events, how='left', on='device_id')

tmp = tmp[['device_id', 'age', 'M_F', 'group_e', 'brand', 'device']]

In [19]:
brand_mf = pd.DataFrame()
brand_mf['count'] = tmp.groupby(['brand', 'M_F'])['M_F'].count()
brand_mf = brand_mf.reset_index()

brand_sum = brand_mf.groupby('brand')['count'].sum()
brand_mf = brand_mf.join(brand_sum, on = 'brand', rsuffix='_')
brand_mf['brand_ratio'] = brand_mf['count'] / brand_mf['count_']
brand_mf = brand_mf.drop(['count', 'count_'], 1)
brand_mf = brand_mf.set_index(['brand', 'M_F'])
brand_mf = brand_mf.unstack()

In [20]:
device_mf = pd.DataFrame()
device_mf['count'] = tmp.groupby(['device', 'M_F'])['M_F'].count()
device_mf = device_mf.reset_index()

device_sum = device_mf.groupby('device')['count'].sum()
device_mf = device_mf.join(device_sum, on = 'device', rsuffix='_')
device_mf['device_ratio'] = device_mf['count'] / device_mf['count_']
device_mf = device_mf.drop(['count', 'count_'], 1)
device_mf = device_mf.set_index(['device', 'M_F'])
device_mf = device_mf.unstack()

In [21]:
brand_g = pd.DataFrame()
brand_g['count'] = tmp.groupby(['brand', 'group_e'])['group_e'].count()
brand_g = brand_g.reset_index()

brand_sum = brand_g.groupby('brand')['count'].sum()
brand_g = brand_g.join(brand_sum, on = 'brand', rsuffix='_')
brand_g['bran_g_ratio'] = brand_g['count'] / brand_g['count_']
brand_g = brand_g.drop(['count', 'count_'], 1)
brand_g = brand_g.set_index(['brand', 'group_e'])
brand_g = brand_g.unstack()

In [22]:
device_g = pd.DataFrame()
device_g['count'] = tmp.groupby(['device', 'group_e'])['group_e'].count()
device_g = device_g.reset_index()

device_sum = device_g.groupby('device')['count'].sum()
device_g = device_g.join(device_sum, on = 'device', rsuffix='_')
device_g['devi_g_ratio'] = device_g['count'] / device_g['count_']
device_g = device_g.drop(['count', 'count_'], 1)
device_g = device_g.set_index(['device', 'group_e'])
device_g = device_g.unstack()

In [24]:
brand_mf = brand_mf.reset_index()
device_mf = device_mf.reset_index()
brand_g = brand_g.reset_index()
device_g = device_g.reset_index()

In [25]:
tmp = pd.merge(tmp, brand_mf, how='left', on='brand')
tmp = pd.merge(tmp, device_mf, how='left', on='device')
tmp = pd.merge(tmp, brand_g, how='left', on='brand')
tmp = pd.merge(tmp, device_g, how='left', on='device')

C:\Python36\lib\site-packages\pandas\core\reshape\merge.py:551: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
C:\Python36\lib\site-packages\pandas\core\reshape\merge.py:862: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  self.right = self.right.drop(right_drop, axis=1)


In [92]:
tmp.head()

,device_id,age,M_F,group_e,brand,device,"(brand_ratio, 0.0)","(brand_ratio, 1.0)","(device_ratio, 0.0)","(device_ratio, 1.0)","(bran_g_ratio, 0.0)","(bran_g_ratio, 1.0)","(bran_g_ratio, 2.0)","(bran_g_ratio, 3.0)","(bran_g_ratio, 4.0)","(bran_g_ratio, 5.0)","(bran_g_ratio, 6.0)","(bran_g_ratio, 7.0)","(bran_g_ratio, 8.0)","(bran_g_ratio, 9.0)","(bran_g_ratio, 10.0)","(bran_g_ratio, 11.0)","(devi_g_ratio, 0.0)","(devi_g_ratio, 1.0)","(devi_g_ratio, 2.0)","(devi_g_ratio, 3.0)","(devi_g_ratio, 4.0)","(devi_g_ratio, 5.0)","(devi_g_ratio, 6.0)","(devi_g_ratio, 7.0)","(devi_g_ratio, 8.0)","(devi_g_ratio, 9.0)","(devi_g_ratio, 10.0)","(devi_g_ratio, 11.0)"
0,-8076087639492063270,35.0,1.0,10.0,51,749,0.282373,0.717627,0.242489,0.757511,0.028529,0.03191,0.044054,0.051172,0.058799,0.067908,0.070609,0.122662,0.077063,0.124799,0.175053,0.147442,0.028851,0.009299,0.010730,0.021936,0.094182,0.077492,0.021221,0.058417,0.078922,0.048641,0.331426,0.218884
1,-2897161552818060146,35.0,1.0,10.0,51,749,0.282373,0.717627,0.242489,0.757511,0.028529,0.03191,0.044054,0.051172,0.058799,0.067908,0.070609,0.122662,0.077063,0.124799,0.175053,0.147442,0.028851,0.009299,0.010730,0.021936,0.094182,0.077492,0.021221,0.058417,0.078922,0.048641,0.331426,0.218884
2,-8260683887967679142,35.0,1.0,10.0,51,749,0.282373,0.717627,0.242489,0.757511,0.028529,0.03191,0.044054,0.051172,0.058799,0.067908,0.070609,0.122662,0.077063,0.124799,0.175053,0.147442,0.028851,0.009299,0.010730,0.021936,0.094182,0.077492,0.021221,0.058417,0.078922,0.048641,0.331426,0.218884
3,-4938849341048082022,30.0,1.0,9.0,51,1524,0.282373,0.717627,0.379536,0.620464,0.028529,0.03191,0.044054,0.051172,0.058799,0.067908,0.070609,0.122662,0.077063,0.124799,0.175053,0.147442,0.020628,0.027677,0.048997,0.065692,0.077193,0.139349,0.063961,0.098192,0.055428,0.084811,0.115382,0.202691
4,245133531816851882,30.0,1.0,9.0,51,753,0.282373,0.717627,0.352371,0.647629,0.028529,0.03191,0.044054,0.051172,0.058799,0.067908,0.070609,0.122662,0.077063,0.124799,0.175053,0.147442,0.060376,0.049907,0.044600,0.035398,0.080826,0.081264,0.085695,0.136576,0.060960,0.121774,0.120800,0.121823


# train / test 분리

In [44]:
train = pd.merge(tmp, gender_age_train, on = 'device_id')
test = pd.merge(tmp, gender_age_test, on = 'device_id')

X = train.copy()

y = train.group_e_x

X = X.drop(['device_id', 'age_x','M_F_x', 'gender', 'age_y', 'group', 'M_F_y', 'group_e_y', 'group_e_x', 'brand', 'device'], 1)

test_e = test.drop(['device_id', 'age', 'M_F', 'group_e', 'brand', 'device'], 1)

X = X.fillna(X.mean())

test_e = test_e.fillna(test_e.mean())

## train_test_split

In [76]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

device_id = test["device_id"].values

# XGBoost

In [50]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

params = {
    'objective' : 'multi:softprob',
    'num_class' : 12,
    'booster' : 'gblinear',
    'max_depth' : 10,
    'eval_metric' : 'mlogloss',
    'eta' : 0.07,
    'silent' : 1,
    'alpha' : 3
}

watchlist = [(dtrain, 'train'), (dtest, 'test')]


gbm = xgb.train(params, dtrain, 40, evals = watchlist, early_stopping_rounds=25, verbose_eval=True)

In [89]:
y_pred = gbm.predict(xgb.DMatrix(test_e))

result = pd.DataFrame(y_pred, columns = le_g.classes_)
result['device_id'] = device_id
#result = result.drop_duplicates(subset=['device_id'], keep='first')

result = result.set_index('device_id')

result.to_csv('subm.csv')

result.head()